In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 113.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForMaskedLM
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/NLP project/Gender_specific_data')

# Preparing Dataset

In [ ]:
#Define the dataset class
class TripleDataset(Dataset):
  def __init__(self, triples):
    self.triples = triples
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

  def __len__(self):
    return len(self.triples)

  def __getitem__(self,idx):
    triple = self.triples.iloc[idx]
    subject,relation,object = eval(triple)

    relation = relation.strip(',')
    subject = subject.strip(',')
    object = object.strip(',')

    input_text = f"{subject} {relation} {object}"
    masked_input_text = f"[MASK] {relation} {object}"
    # inputs = self.tokenizer(input_text, return_tensor='pt',paddng=True, truncation=True)
    encoded = self.tokenizer.encode_plus(input_text, add_special_tokens=True, padding='max_length', max_length=128,
                                         truncation=True, return_tensors='pt')
    input_ids=encoded['input_ids'].squeeze(0)
    # attention_mask=encoded['attention_mask'].squeeze(0)
    # token_type_ids=encoded['token_type_ids'].squeeze(0)

    encoded2 = self.tokenizer.encode_plus(masked_input_text, add_special_tokens=True, padding='max_length', max_length=128,
                                         truncation=True, return_tensors='pt')
    masked_input_ids=encoded2['input_ids'].squeeze(0)
    attention_mask=encoded2['attention_mask'].squeeze(0)
    token_type_ids=encoded2['token_type_ids'].squeeze(0)

    return {'labels': input_ids,
            'input_ids': masked_input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
            'subject': subject,
            'object': object
            }

In [ ]:
# def collate_fn(batch):
#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#     input_ids = [item['input_ids'] for item in batch]
#     masked_input_ids = [item['masked_input_ids'] for item in batch]
#     attention_mask = [item['attention_mask'] for item in batch]
#     token_type_ids = [item['token_type_ids'] for item in batch]
#     subjects = [item['subject'] for item in batch]
#     objects = [item['obj'] for item in batch]

#     input_ids = pad_sequence(input_ids, batch_first=True)
#     attention_mask = pad_sequence(attention_mask, batch_first=True)
#     token_type_ids = pad_sequence(token_type_ids, batch_first=True)

#     mask_token_id = tokenizer.mask_token_id
#     labels = input_ids.clone()

#     for i, subject in enumerate(subjects):
#         subject_start = (input_ids[i] == tokenizer.encode('[CLS]', add_special_tokens=False)[0]).nonzero(as_tuple=False)[0]
#         subject_end = (input_ids[i] == tokenizer.encode('[SEP]', add_special_tokens=False)[0]).nonzero(as_tuple=False)[-1] + 1
#         subject_tokens = tokenizer.encode(subject, add_special_tokens=False)
#         input_ids[i, subject_start:subject_end] = mask_token_id
#         labels[i, subject_start:subject_end] = input_ids[i, subject_start:subject_end]

#     return {
#         'input_ids': input_ids,
#         'attention_mask': attention_mask,
#         'token_type_ids': token_type_ids,
#         'labels': labels,
#         'subjects': subjects,
#         'objects': objects
#     }


In [ ]:
# make the dataloader
triples = data['triple']
dataset = TripleDataset(triples)
dataloader = DataLoader(dataset, batch_size=32)

In [ ]:
tokenizer.decode(dataset[100]['input_ids'])

'[CLS] [MASK] father wossen seged [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
#@title Default title text
dataset[30]

{'labels': tensor([  101, 19542,  7274,  2271,  2388, 29347,  9354,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0

In [ ]:
# Soft Prompt Configuration
soft_prompt = torch.randn(1, 128, 768)  # Replace with your desired soft prompt

# Define the model

In [ ]:
# Knowledge Prompt (KP) Model
class KPModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(KPModel, self).__init__()
        self.hidden_layer = torch.nn.Linear(input_dim, hidden_dim)
        self.activation = torch.nn.ReLU()
        self.output_layer = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.hidden_layer(x)
        x = self.activation(x)
        x = self.output_layer(x)
        return x


In [ ]:
# Language Model
lm_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
for param in lm_model.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device:',device)
kp_model = KPModel(input_dim=32, hidden_dim=512, output_dim=384)
kp_model.to(device)

optimizer = torch.optim.AdamW(kp_model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

num_epochs=5


device: cpu


In [ ]:
# Soft Prompt Configuration
soft_prompt = torch.randn(1, 1, 768)  # Soft prompt with dimensions (1, 1, 768)

# Training the KP

In [ ]:
for batch in dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    labels = batch['labels'].to(device)
    subjects = batch['subject']
    objects = batch['object']

    optimizer.zero_grad()

    for subject in subjects:
        subject_tokens = tokenizer.encode(subject, add_special_tokens=False)  # Encode the subject

        # Convert subject_tokens to a tensor
        subject_tensor = torch.tensor(subject_tokens, dtype=torch.float32).unsqueeze(0)  # Unsqueeze to add a batch dimension

        # Repeat subject_tensor to match the batch size of input_ids
        subject_tensor = subject_tensor.repeat(input_ids.size(0), 8)

        # Pass the subject to the KP model
        kp_outputs = kp_model(subject_tensor)

        # Concatenate input_ids with kp_outputs_repeated
        print(input_ids.size())
        print(kp_outputs.size())
        input_with_kp = torch.cat([input_ids, kp_outputs.to(input_ids.dtype)], dim=-1)

        lm_model.eval()

        # Pass the input with KP to the language model
        print(input_with_kp.size())
        attention_mask = attention_mask.float()
        outputs = lm_model(input_ids=input_with_kp, attention_mask=attention_mask, token_type_ids=token_type_ids)

        lm_model.train()

        # Calculate loss based on the predicted subject/object and the actual subject/object
        predicted_subjects = outputs.logits[:, :-1, :][attention_mask[:, :-1] == 1]
        predicted_objects = outputs.logits[:, 1:, :][attention_mask[:, 1:] == 1]

        subject_loss = criterion(predicted_subjects, subjects)
        object_loss = criterion(predicted_objects, objects)

        kp_loss = subject_loss + object_loss

        total_kp_loss += kp_loss.item() * input_ids.size(0)
        total_examples += input_ids.size(0)

        kp_loss.backward()
        optimizer.step()

avg_kp_loss = total_kp_loss / total_examples
print(f"Epoch: {epoch + 1}/{num_epochs}, KP Loss: {avg_kp_loss:.4f}")
